## ANALISIS TRACK BASADO EN PWD

In [ ]:
#PASO_1
#Importamos un xml procedente de pwd

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd

path='Mallorca70.3_tp'
DOMTree = xml.dom.minidom.parse(path+".xml")


collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("sample")
list_track = []

count=0
for trkpt in trkpts:
    count +=1
    if len(trkpt.getElementsByTagName("timeoffset"))==0:
        timeoffset = ''
    else:
        timeoffset = float(trkpt.getElementsByTagName("timeoffset")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("spd"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("spd")[0].firstChild.nodeValue)    
    
    if len(trkpt.getElementsByTagName("pwr"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("pwr")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("cad"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("cad")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("dist"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("dist")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lat"))==0:
        lat = ''
    else:
        lat = float(trkpt.getElementsByTagName("lat")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lon"))==0:
        lon = ''
    else:
        lon = float(trkpt.getElementsByTagName("lon")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("alt"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("alt")[0].firstChild.nodeValue)    

    if len(trkpt.getElementsByTagName("temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("temp")[0].firstChild.nodeValue)
   
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

print ("Puntos cargados: %d" %count)

    

result = pd.DataFrame(list_track)
result.columns=["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
result.to_excel('Intermedio'+path+'.xlsx')
result.to_excel(path+".xlsx")
print('finish')



## ANALISIS TRACK BASADO EN GPX

In [5]:
#PASO_1
#Importamos un xml procedente de gpx

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd


#path='Analisis_Aerodinamica'
#path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'
path='20180406_LEVS_195_90_R_E'
DOMTree = xml.dom.minidom.parse(path+".xml")

collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("trkpt")
list_track = []

def to_dt(dato):
    t=pd.to_datetime(dato)
    return t.hour*3600+t.minute*60+t.second

count=0

for trkpt in trkpts:
    count +=1
    if trkpt.hasAttribute("lat"):
      lat = float(trkpt.getAttribute("lat"))
    if trkpt.hasAttribute("lon"):
      lon = float(trkpt.getAttribute("lon"))
    
    if len(trkpt.getElementsByTagName("ele"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("ele")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("time"))==0:
        time = ''
    else:
        time = trkpt.getElementsByTagName("time")[0].firstChild.nodeValue
    
    if len(trkpt.getElementsByTagName("gpxdata:hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("gpxdata:hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:cadence"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("gpxdata:cadence")[0].firstChild.nodeValue)
        
    if len(trkpt.getElementsByTagName("gpxdata:temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("gpxdata:temp")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:distance"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("gpxdata:distance")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:power"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("gpxdata:power")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("gpxdata:speed"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("gpxdata:speed")[0].firstChild.nodeValue)
    
    timeoffset = to_dt(time)
    
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

result = pd.DataFrame(list_track)
result.columns = ["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
print ("Puntos cargados: %d" %count)
result.to_excel('Intermedio'+path+'.xlsx')
print('finish')


Puntos cargados: 7586
finish


In [ ]:
#conda install -c conda-forge geopy

## ENRIQUECEMOS EL FICHERO. COMUN PARA TODOS LOS ORIGENES

In [6]:
import pandas as pd
import math as mt
from scipy.signal import savgol_filter

In [2]:
def func_bearing(bc,bv):
    if bv+180 > 360:
        bv_mod = bv-180
    else:
        bv_mod = bv+180
    if bc > bv_mod:
        dif = bc-bv_mod
    else:
        dif = bv_mod-bc
        
    multiplicador_viento = mt.cos(dif/(180/mt.pi))
    #return(bc, bv, bv_mod, dif, multiplicador_viento)
    return(multiplicador_viento, dif)

def sync_bearing(ori_bearing):
    if ori_bearing < -90:
        new_bearing = 450+ori_bearing
    else:
        new_bearing = ori_bearing+90
    return new_bearing

def deteccion_tramos(input_df):
    tramos = [0]
    tramo = 0
    pendiente = 0
    pendiente_ant = 0

    tramos_100 = [0]
    tramo100 = 0
    acum_dist = 0
        

    lista_tramos = [(0,0,"Inicio")]

    lista_tramos_2 = []
    tramo_inicial = 0
    power_acum = 0
    
    i = 1
    input_df.iloc[0,5] = 0
    
    
    print("Entramos en el While1 %d" % len(input_df.index))
    while i < len(input_df.index):
        
        
        power_acum = power_acum + (input_df.iloc[i]["pwr"]*(input_df.iloc[i]["timeoffset"]-input_df.iloc[i-1]["timeoffset"]))
        
        #Calculo de tramos continuos
        
        #print("punto = %d y tramo = %d" % (i, tramo))
        pendiente = input_df.iloc[i]["new_ele"]-input_df.iloc[i-1]["new_ele"]

        if pendiente * pendiente_ant >=0:
            tramos.append(tramo)
        else:
            #Cada tramo en 1 línea
            distancia_tramo = input_df.iloc[i-1]["dist"]-input_df.iloc[tramo_inicial]["dist"]
            tiempo_tramo = input_df.iloc[i-1]["timeoffset"]-input_df.iloc[tramo_inicial]["timeoffset"]
            desnivel_tramo = input_df.iloc[i-1]["new_ele"]-input_df.iloc[tramo_inicial]["new_ele"]
            porcent_des = desnivel_tramo/distancia_tramo*100
            sp_pwd = power_acum/tiempo_tramo
            vel_calc = (distancia_tramo/tiempo_tramo)*3.6
            lista_tramos_2.append((tramo, tramo_inicial, i-1,sp_pwd,tiempo_tramo,distancia_tramo,vel_calc,porcent_des))
            
            power_acum = 0
            
            #Cada Inicio o Fin de tramo en 1 línea
            tramo_inicial = i
            lista_tramos.append((tramo,i-1,"Fin"))
            tramo +=1
            tramos.append(tramo)
            lista_tramos.append((tramo,i,"Inicio"))
            
        pendiente_ant = pendiente            
        
        #Calculo de tramos de 100m
        acum_dist = input_df.iloc[i-1]["dist"]
        tramo100 = mt.floor(acum_dist/100)
        tramos_100.append(tramo100)
        
        i +=1
          


    print("tramos : %d continuos y %d de 100m" % (len(set(tramos)), len(set(tramos_100))))
    print("Salimos del While")
   


    return lista_tramos_2, lista_tramos, tramos, tramos_100

In [8]:
path='20180406_LEVS_195_90_R_E'

df = pd.read_excel('Intermedio'+path+'.xlsx')

In [9]:
df

,timeoffset,hr,spd,pwr,cad,dist,lat,lon,alt,temp,prueba
0,27865,NaN,5.000000,0.000000,NaN,NaN,40.519800,-3.929805,662.799988,22.700001,20180406_LEVS_195_90_R_E
1,27867,NaN,5.141748,0.000000,0.0,15.000000,40.519850,-3.929899,661.417483,22.700001,20180406_LEVS_195_90_R_E
2,27868,NaN,5.200000,0.000000,0.0,26.000000,40.519883,-3.930019,661.610010,22.600000,20180406_LEVS_195_90_R_E
3,27870,NaN,5.238999,0.000000,0.0,39.000000,40.519872,-3.930170,659.610010,22.600000,20180406_LEVS_195_90_R_E
4,27872,NaN,5.300000,0.000000,0.0,52.000000,40.519832,-3.930314,659.000000,22.600000,20180406_LEVS_195_90_R_E
5,27874,NaN,5.352179,0.000000,0.0,65.000000,40.519736,-3.930410,658.869553,22.600000,20180406_LEVS_195_90_R_E
6,27876,NaN,6.059540,0.000000,0.0,81.000000,40.519628,-3.930525,658.000000,22.600000,20180406_LEVS_195_90_R_E
7,27878,NaN,6.338999,0.000000,0.0,93.000000,40.519561,-3.930643,658.000000,22.600000,20180406_LEVS_195_90_R_E
8,27880,NaN,6.400000,0.000000,0.0,106.000000,40.519532,-3.930786,658.000000,22.600000,20180406_LEVS_195_90_R_E
9,27882,NaN,6.400000,0.000000,0.0,124.706971,40.519534,-3.931000,657.000000,22.600000,20180406_LEVS_195_90_R_E


In [7]:
#path='Analisis_Aerodinamica'
#path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'
path='20180406_LEVS_195_90_R_E'

df = pd.read_excel('Intermedio'+path+'.xlsx')

df2 = df[df['lat'] >0]
df2['pwr'].fillna(0, inplace=True)
#df2.drop(['dist','cad','hr','spd'], axis=1, inplace=True)

from geopy.distance import distance
from geopy import Point

i=1
bv=0

wind=[]
wind.append(0)

mult=[]
mult.append(0)

dist_c=[]
dist_c.append(0)

ele_gain = []
ele_gain.append(0)

d=0
e=0

porc=[]
porc.append(0)

spd_c=[]
spd_c.append(0)

spd_prev=[]
spd_prev.append(0)

ele_prev=[]
ele_prev.append(0)

bearing=[]
bearing.append(0)

timeoffset_prev=[]
timeoffset_prev.append(0)

ini_timeoffset = df2.iloc[0]['timeoffset']
time=[]
time.append(0)


while i < len(df2):
    #print("Punto %d" %i
    punto1 = Point(df2.iloc[i-1]['lon'], df2.iloc[i-1]['lat'])
    punto2 = Point(df2.iloc[i]['lon'], df2.iloc[i]['lat'])
    time.append(df2.iloc[i]['timeoffset']-ini_timeoffset)
    dist_c.append(distance(punto1, punto2).m)
    d = distance(punto1, punto2).m
    spd_c.append(float(d/(df2.iloc[i]['timeoffset']-df2.iloc[i-1]['timeoffset'])))
    punto1 = df2.iloc[i-1]['alt']
    punto2 = df2.iloc[i]['alt']
    ele_gain.append((punto2)-(punto1))
    e = punto2-punto1
    porc.append(e*100/d)
    lona = df2.iloc[i-1]['lon']
    lata = df2.iloc[i-1]['lat']
    lonb = df2.iloc[i]['lon']
    latb = df2.iloc[i]['lat']
    X = mt.cos(lonb)*mt.sin(latb-lata)
    Y = mt.cos(lona)*mt.sin(lonb)-mt.sin(lona)*mt.cos(lonb)*mt.cos(latb-lata)
    
    #bearing.append(mt.atan2(X,Y)*57.2957795)  Se modifica por la siguiente línea que corrige estandariza el ángulo
    bearing.append(sync_bearing(mt.atan2(X,Y)*57.2957795))
    mult.append(func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[0])
    wind.append(func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[1])
    
    i+=1
df2['dist_c']=dist_c
df2['ele_gain']=ele_gain
df2['porc']=porc
df2['spd_c']=spd_c
df2['bearing']=bearing
df2['wind']=wind
df2['mult']=mult
i=1
while i < len(df2):
    spd_prev.append(df2.iloc[i-1]['spd_c'])
    ele_prev.append(df2.iloc[i-1]['alt'])
    timeoffset_prev.append(df2.iloc[i-1]['timeoffset'])
    i+=1
df2['spd_prev']=spd_prev
df2['ele_prev']=ele_prev
df2['timeoffset_prev']=timeoffset_prev

#aplicamos el suavizado de señal
axis_y = df["alt"]
axis_yhat = savgol_filter(axis_y, 31, 2)
df2["new_ele"] = axis_yhat
df2["time_seg"] = time

df2["tramo"] = tramos
df2["tramos_100"] = tramos_100


# lista_tramos_2, lista_tramos, tramos, tramos_100 = deteccion_tramos(df2)

#path='20180406_LEVS_195_90_R_E'


#df2.to_excel(path+'_ml.xlsx')

print('finish')


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide


NameError: name 'tramos' is not defined

In [ ]:
df2[0:75]

In [ ]:
lista_tramos_2, lista_tramos, tramos, tramos_100 = deteccion_tramos(df2)
#tramo, tramo_inicial, tramo_final,pwd,tiempo_tramo,distancia_tramo,vel_calc,porcent_des
result = pd.DataFrame(lista_tramos_2)
result.columns= ["tramo", "tramo_inicial", "tramo_final","sp_pwd","tiempo_tramo","distancia_tramo","vel_calc","porcent_des"]
result

In [ ]:
#AÑADIMOS NUEVAS VARIABLES DEPENDIENTES DE LA RUTA Y EL CICLISTA
#altura = path.split('_')[2]
#peso = path.split('_')[3]
#tpo_bici = path.split('_')[4]
#tpo_entreno = path.split('_')[5]

df2['altura'] = path.split('_')[2]
df2['peso'] = path.split('_')[3]
df2['tpo_bici'] = path.split('_')[4]
df2['tpo_entreno'] = path.split('_')[5] 

df2.to_csv(path+'_ml.csv')
print('finish')
#print(altura,peso,tpo_bici,tpo_entreno)
#df2

In [ ]:
df2

In [ ]:
lista_tramos_2